# Lecture 5 : Functions and Pointers

# Part 1 : Primality Testing

## Recall our primality tester from lecture 4.

In [ ]:
%%writefile prime_v3.c
#include <stdio.h>
#include <stdlib.h>

int main (int argc, char** argv) {
    if (argc < 2) {
        printf ("command usage: %s %s\n",argv[0],"n");
        return 1; // abnormal exit
    }
    long long n = atoll(argv[1]);
    if (n % 2 == 0) {
        printf ("The number %lld is not prime since 2 divides it\n",n);
        return 0;
    }
    for (long long d = 3; d*d <= n; d+=2) {
        if (n % d == 0) {
            printf ("The number %lld is not prime since %lld divides it.\n",n,d);
            return 0;
        }
    }
    printf ("The number %lld is prime.\n",n);
}

Overwriting prime_v3.c


## As our programs become more complicated we will want to make use of C functions to make our code more readable and easier to maintain.

## Here is a version of our primality tester where we test to see if the number is prime using a function.

## Note that the syntax for C functions is similar to Java functions.

## Although Java has a built-in boolean type, the original version of C does not.  

## However boolean types were added to C in the C99 standard.  

## We include *stdbool.h* to access this new boolean type in C.

In [ ]:
%%writefile fun_prime_v1.c
#include <stdio.h>
#include <stdlib.h>
#include <stdbool.h>

bool is_prime(long long n) {
    if (n % 2 == 0) {
        return false;
    }
    for (long long d = 3; d*d <= n; d+=2) {
        if (n % d == 0) {
            return false;
        }
    }
    return true;
}

int main (int argc, char** argv) {
    if (argc < 2) {
        printf ("command usage: %s %s\n",argv[0],"n");
        return 1; // abnormal exit
    }
    long long n = atoll(argv[1]);
    if (is_prime(n)) {
        printf ("The number %lld is prime.\n",n);
    } else {
        printf ("The number %lld is not prime.\n",n);
    }
}

Overwriting fun_prime_v1.c


In [ ]:
!gcc -o fun_prime_v1 fun_prime_v1.c

In [ ]:
!./fun_prime_v1 5261656080911617

The number 5261656080911617 is prime.


In [ ]:
!./fun_prime_v1 3439315899953761

The number 3439315899953761 is not prime.


## Note that line 24 is equivalent to
    if (is_prime(n) == true)

## Also note we no longer print a divisor of n when n is not prime.

## In order to add this functionality back in, we need our function *is_prime* to return a second value.

## There are multiple ways to write functions in C that return more than one value.

## One common way is to allow the function to modify one or more of its arguments using **C pointers**.

# Part 2 : Introduction to Pointers in C

## C pointers are memory addresses and can be used to assign values to variables **indirectly**.

## Pointers are not available in Java.  This is one of the major differences between C and Java.

## All variables declared in your C program are stored somewhere in the systems **memory** (this is true for any programming language but we rarely think about how our variables are stored when using high level languages such as Python and R).

## We group memory into 8 bit chunks called **bytes**.

## The number of bytes that variables of each C type occupy in memory is system dependent.  

## We can use the C function **sizeof** to learn how many bytes variables occupy in memory.  

In [115]:
%%writefile sizeof.c
#include <stdio.h>

int main () {
    int a;
    long long b;
    float c;
    double d;
    printf ("the variable a of type int occupies %lu bytes in memory\n",sizeof(a));
    printf ("the variable b of type long long occupies %lu bytes in memory\n",sizeof(b));
    printf ("the variable c of type float occupies %lu bytes in memory\n",sizeof(c));
    printf ("the variable d of type double occupies %lu bytes in memory\n",sizeof(d));

}

Overwriting sizeof.c


In [116]:
!gcc -o sizeof sizeof.c

In [117]:
!./sizeof

the variable a of type int occupies 4 bytes in memory
the variable b of type long long occupies 8 bytes in memory
the variable c of type float occupies 4 bytes in memory
the variable d of type double occupies 8 bytes in memory


## How many **bits** does each variable occupy in memory?

## In C we can go a significant step further and figure out **where** our variables are stored in memory!

## To do this we use the C *address of* operator **&**.

## If we think of memory as an array of bytes, then the address of a variable is just the index in that array where the first byte of the variable is stored.

## Note that addresses in C are normally printed in hexadecimal using the %p format specifier.  

## In the code below we instead print the addresses in decimal for clarity.

In [125]:
%%writefile address.c
#include <stdio.h>

int main () {
    int a = 0;
    int b = 16;
    int c = 257;
    printf ("the first byte of a is stored at memory address %lu\n",(unsigned long)&a);
    printf ("the first byte of b is stored at memory address %lu\n",(unsigned long)&b);
    printf ("the first byte of c is stored at memory address %lu\n",(unsigned long)&c);
}

Overwriting address.c


In [123]:
!gcc -o address address.c

In [124]:
!./address

the first byte of a is stored at memory address 140731662434508
the first byte of b is stored at memory address 140731662434512
the first byte of c is stored at memory address 140731662434516


## Note that the addresses are very large!  

## Most modern computers use 48 bits of address space which is enough to specify around 281 trillion unique memory locations.

## Next let's snoop into memory to see **exactly** how our variables are stored as sequences of bytes.  

In [133]:
%%writefile snoop.c
#include <stdio.h>

int main () {
    int a = 0;
    int b = 16;
    int c = 257;
    printf ("the first byte of a is stored at memory address %lu\n",(unsigned long)&a);
    printf ("the first byte of b is stored at memory address %lu\n",(unsigned long)&b);
    printf ("the first byte of c is stored at memory address %lu\n",(unsigned long)&c);
    printf ("here are the 12 bytes of memory starting at the address of a:\n");
    for (int i=0;i<12;i++) {
        printf ("%u ",((unsigned char*)&a)[i]);
    }
    printf ("\n");
}

Overwriting snoop.c


In [134]:
!gcc -o snoop snoop.c

In [135]:
!./snoop

the first byte of a is stored at memory address 140726712305080
the first byte of b is stored at memory address 140726712305084
the first byte of c is stored at memory address 140726712305088
here are the 12 bytes of memory starting at the address of a:
0 0 0 0 16 0 0 0 1 1 0 0 


## The variable *a* is initialized to 0.  Each byte of the 4 bytes required for *a* are equal to 0.

## The variable *b* is initialized to 16.  

## Since the value 16 is small enough to fit into a single byte, the remaining 3 bytes of $b$ are 0.

## Note that *b* is stored least significant byte first.  

## This byte ordering is called **little endian** (most modern computers use little endian ordering).

## The variable *c* is initialized to 257.

## The first and second bytes of *c* are both 1 since 257 = 1 + 256.

## Now that we understand the basics of how variables are stored in memory, we are ready to learn about C **pointers**.

## What value do you think the program below will output?

In [130]:
%%writefile mystery.c
#include <stdio.h>

int main () {
    int a = 3;
    int* p = &a;
    *p = 4;
    printf ("a = %d\n",a);
}

Writing mystery.c


In [131]:
!gcc -o mystery mystery.c

In [132]:
!./mystery

a = 4


## Here are some notes to explain this output!

* ### First note in line 6 that the type of the variable *p* is *int**.  In other words, $p$ is a **pointer** to an integer.  

* ### A C **pointer** is a variable that stores the **memory address** of another variable.

* ### Recall that & is the C operator that takes the **address** of a variable.  In particular, in line 6 the &a is the address (or location) where the variable *a* is stored in memory.

* ### Putting these two ideas together, in line 6 we **declare** the variable *p* to be a pointer to an integer and we **initialize** the variable *p* to the memory address of the variable *a*.  

* ### In other words, line 6 initializes the integer pointer *p* to **point** to the integer variable *a*.

* ### In line 7, the * is the C operator that **dereferences** a pointer.  In other words, **p* is an alias for the variable that the pointer *p* points to which is the variable *a*.  Thus, line 7 assigns the value 4 to the variable *a*.  

## More succinctly, we changed the value of $a$ to $4$ indirectly using the pointer $p$!

## At this point you may be wondering why someone would want to use C pointers.  After all line 7 could be replaced with $a=4$.  This change would not effect the output of the program and the code would certainly be easier to read and understand.  

## The benefits (and dangers) of using C pointers will become more clear as we progress.  

## To understand our first application of C pointers we need to learn about **C function call semantics**.


# Part 3 : C Function call Semantics

## Consider the following C code that attempts to add 2 integers.

## What value do you think the program will output?

In [ ]:
%%writefile sum2_v1.c
#include <stdio.h>

void add(int a, int b, int c) {
    c = a + b;
}

int main () {
    int sum = 0;
    add(3,7,sum);
    printf ("sum = %d\n",sum);
}

Overwriting sum2_v1.c


In [ ]:
!gcc -o sum2_v1 sum2_v1.c

In [ ]:
!./sum2_v1

sum = 0


## The code outputs 0 which is the initial value of sum assigned in line 9 instead of the sum of 3 and 7 which is 10.   

## The call to the function *add* did not change the variable *sum*.

## **Like Java, C passes arguments of basic types such as *int* and *float* to functions by value (also called pass by copy).**

## When the function *add* is called, the values 3, 7, and the value of sum are copied to the *local* variables a, b, and c (we call a, b, c *local* variables because they can only be accessed by the *sum* function).    

## Since *c* is a copy of *sum*, the value of the variable *sum* is not changed by the function *add*.

## In C we can work around the pass by value semantics by passing a **pointer to the sum variable** instead of the value of the sum variable.  Using a pointer to sum, the *add* function will be able to change the value of sum indirectly (just like we changed *a* indirectly using the *p* pointer in the previous example).    

## Here is the new version that gives the expected output:

In [ ]:
%%writefile sum2_v2.c
#include <stdio.h>

void add(int a, int b, int* c) {
    *c = a + b;
}

int main () {
    int sum = 0;
    add(3,7,&sum);
    printf ("sum = %d\n",sum);
}

Overwriting sum2_v2.c


In [ ]:
!gcc -o sum2_v2 sum2_v2.c

In [ ]:
!./sum2_v2

sum = 10


## Notes:

* ### In line 4 we declare the third argument c to be an integer pointer.

* ### In line 5 we dereference the pointer c using the syntax **c* so that the variable that the pointer c points to (which is the variable sum in this example) is assigned the value of *a+b*.

* ### In line 10 we pass a pointer to the sum as our third argument using the *&sum* syntax (recall that & is the address operator in C).

# Part 4 : Primality Testing Revisited

## Here is a version of the primality tester with a function that prints a divisor of n if n is not prime.

In [ ]:
%%writefile fun_prime_v2.c
#include <stdio.h>
#include <stdlib.h>
#include <stdbool.h>

bool is_prime(long long n, long long* divisor) {
    if (n % 2 == 0) {
        *divisor = 2;
        return false;
    }
    for (long long d = 3; d*d <= n; d+=2) {
        if (n % d == 0) {
            *divisor = d;
            return false;
        }
    }
    return true;
}

int main (int argc, char** argv) {
    if (argc < 2) {
        printf ("command usage: %s %s\n",argv[0],"n");
        return 1; // abnormal exit
    }
    long long n = atoll(argv[1]);
    long long divisor;
    if (is_prime(n,&divisor)) {
        printf ("The number %lld is prime.\n",n);
    } else {
        printf ("The number %lld is not prime since %lld divides it.\n",n,divisor);
    }
}

Overwriting fun_prime_v2.c


In [ ]:
!gcc -o fun_prime_v2 fun_prime_v2.c

In [ ]:
!./fun_prime_v2 5261656080911617

The number 5261656080911617 is prime.


In [ ]:
!./fun_prime_v2 3439315899953761

The number 3439315899953761 is not prime since 58645681 divides it.


# Part 5 : Reading Input from *stdin* Using *scanf*

## In lecture 4 we learned how to use command line arguments to get input from the user at runtime.  

## Command line arguments only make sense for a small amount of input.  

## For larger inputs (such as a file containing data) we can read the input from *stdin* using *scanf* (formatted scan).  

## Arguments to *scanf* are passed by pointer so that the variables they point to can be modified (indirectly) to contain the next inputs from *stdin*.  

## The return value of *scanf* gives the number of fields that were successfully converted and assigned to variables.

## One key use of the return value of *scanf* is to check for *end of file*.

### If scanf reaches the end of the *stdin* input stream (i.e. end of file), *scanf* will return EOF (a negative number constant defined inside stdio.h) and the loop exits.

## Here is a C program that sums the numbers read from *stdin*.  

In [ ]:
%%writefile sum.c
#include <stdio.h>

int main () {
    int sum = 0;
    int next;
    while (scanf("%d",&next) == 1) {
        sum = sum + next;
    }
    printf ("sum = %d\n",sum);
}

Overwriting sum.c


In [ ]:
!gcc -o sum sum.c

In [ ]:
!echo 10 20 30 | ./sum

sum = 60


In [ ]:
!echo 5 10 15 20 > num.dat

In [ ]:
!cat num.dat | ./sum

sum = 50


## Here is a function that finds the smallest number in the input stream *stdin*.

In [ ]:
%%writefile smallest.c
#include <stdio.h>
#include <limits.h>

int main () {
    int smallest = INT_MAX;
    int next;
    while (scanf("%d",&next) == 1) {
        if (next < smallest) {
            smallest = next;
        }
    }
    printf ("The smallest number is %d\n",smallest);
}

Overwriting smallest.c


In [ ]:
!gcc -o smallest smallest.c

In [ ]:
!echo -25 50 -75 100 -200 -3433 4500 | ./smallest

The smallest number is -3433


## Exercise 1 : Write a program called *smallest3* that finds the three smallest integers in *stdin*.

## Hint: Copy smallest.c to smallest3.c and modify it.

## Here is a program that finds the integer in *stdin* closest to a given number.

In [ ]:
%%writefile closest.c
#include <stdio.h>
#include <stdlib.h>
#include <limits.h>

int main (int argc, char* argv[]) {
    if (argc < 2) {
        printf ("command usage: %s %s\n",argv[0],"center");
    }
    int center = atoi(argv[1]);
    int closest;
    int smallest_distance = INT_MAX;
    int next;
    while (scanf("%d",&next) == 1) {
        int distance = abs(next-center);
        if (distance < smallest_distance) {
            closest = next;
            smallest_distance = distance;
        }
    }
    printf ("The number closest to %d is %d\n",center,closest);
}

Overwriting closest.c


In [ ]:
!gcc -o closest closest.c -lm

In [ ]:
!echo 15 -7 25 -2 30 -5 29 37 -25 | ./closest 5

The number closest to 5 is -2


## Exercise 2 : Write a program called *closest3* that finds the three integers in *stdin* closest to a given command line argument *center*.

## Hint: Copy closest.c to closest3.c and modify it.